In [1]:
import os

os.environ['PYTHONPATH']

'.;C:\\Users\\Playdata\\코딩\\ObjectDetection\\models;C:\\Users\\Playdata\\코딩\\ObjectDetection\\models\\research'

In [2]:
import object_detection

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [4]:
PIPELINE_CONFIG_PATH = os.path.join('model', 'pipeline.config')
CHECK_POINT_PATH = os.path.join('model', 'checkpoint', 'ckpt-21')
LABEL_MAP_FILE_PATH = os.path.join('labelmap', 'label_map.pbtxt')

In [5]:
# pipeline.config에 맞춰서 추출한 모델을 바탕으로 모델을 생성

# pipeline.config를 조회
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
# pipeline.config의 model 설정 정보를 넣어 모델 생성
detection_model = model_builder.build(model_config=config['model'], is_training=False)

# 모델에 학습시킨 checkpoint(weight)를 주입
# checkpoint 조회
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECK_POINT_PATH)).expect_partial()

In [9]:
# detection을 실행하는 함수

# 순전파처리 함수에 @tf.function decorator를 선언하면 실행 속도가 빨라짐
@tf.function
def detect_func(image):
    """
    매개변수로 object detection을 수행할 대상 image(Tensor)를 받아서 detection 처리
    1. preprocessing(전처리): resize, normalization 작업
    2. detection(inference-추론)
    3. detection 결과를 post processing : Non Maximum Suppression
    4. post processing한 결과 반환
    """
    # 1. preprocessing
    image, shapes = detection_model.preprocess(image)
    # 2. 추론
    predict_dict = detection_model.predict(image, shapes)
    # 3. post processing
    result = detection_model.postprocess(predict_dict, shapes)

    return result

In [15]:
# 웹캠으로부터 이미지를 받아서 추론한 결과를 화면에 보여주기
cap = cv2.VideoCapture(0)
# 웹캠의 WIRDTH/HEIGHT 조회
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_FILE_PATH)

In [16]:
while True:
    ret, frame = cap.read()  #한 frame 읽기
    if not ret:
        print("이미지를 읽지 못 했습니다")
        break
        
    frame = cv2.flip(frame, 1)  #좌우 반전
    
    # BGR => RGB (모델이 학습할 때 RGB 모드로 학습했기 때문에 같은 형식으로 변환)
    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # 0번 축을 늘리고 Tensor로 변환
    input_tensor = tf.convert_to_tensor(image_np[np.newaxis, ...], dtype=tf.float32)
    
    # 추론
    post_detection = detect_func(input_tensor)  #전처리-추론-후처리
    
    num_detections = int(post_detection.pop('num_detections'))
    # 추론한 결과들을 num_detections 개수(detection한 물체의 개수)만큼의 값만 남긴다. 결과가 Tensor로 반환되는 것을 ndarray로 변환
    detections = {key:value[0, :num_detections].numpy() for key, value in post_detection.items()}

    # 새로 구성한 결과 dictionary(detections)에 num_detections 값을 추가
    detections['num_detections'] = num_detections
    # detection_classes는 검출한 box의 class 값을 label encoding된 값으로 가진다. float32로 반환되는 것을 int로 변환 처리
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    MIN_CONF_THRESH = 0.5  #물체가 있을 Confidence score가 0.5 이상인 bounding box만 나오도록 함
    image_np_with_detection = image_np.copy()  #detection한 원본 이미지의 카피본 생성
    img = viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detection,  #추론할 원본 이미지,
                    detections['detection_boxes'],  #bounding box 좌표
                    detections['detection_classes'] + 1,  #bounding box 내의 물체 index(class확률에서 0은 첫번째 label, label map의 id는 1부터 시작하기 때문에 +1)
                    detections['detection_scores'],  #bounding box 내의 물체가 있을 확률(confidence score)
                    category_index,
                    use_normalized_coordinates=True,  #bounding box의 좌표들이 normalize되었는지 여부
                    max_boxes_to_draw=200,  #최대 몇개 박스를 칠 것인지(기본: 20)
                    min_score_thresh=MIN_CONF_THRESH  #Confidence score가 얼마 이상인 bounding box만 나오도록 하겠다
                    )

    # 결과 image를 RGB => BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # 화면에 출력
    cv2.imshow('frame', img)
    if cv2.waitKey(0) == 27:  #esc를 입력하면
        break
        
cap.release()
cv2.destroyAllWindows()
    

## post processing 결과 확인

In [17]:
img = cv2.imread('one.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img[np.newaxis, ...]
input_tensor = tf.convert_to_tensor(img, dtype=tf.float32)

In [18]:
post_detection = detect_func(input_tensor)

In [19]:
post_detection.keys()

dict_keys(['detection_boxes', 'detection_scores', 'detection_classes', 'num_detections', 'raw_detection_boxes', 'raw_detection_scores', 'detection_multiclass_scores', 'detection_anchor_indices'])

In [21]:
post_detection['num_detections'].numpy()  #Tensor => ndarray
# num_detections: 후처리 결과로 나온 bounding box의 개수. 전체 bounding box에서 confidence score순으로 내림차순한 뒤 NMS를 거쳐서 최종적으로 남은 bounding box의 개수

array([100.], dtype=float32)

In [22]:
post_detection['detection_boxes'].shape
# [1, 100, 4] => [추론한 이미지 개수, num_detections, 좌표]

TensorShape([1, 100, 4])